# Computing the nonadiabatic couplings in Kohn-Sham and excited states bases

In this tutorial, we will start computing the nonadiabatic couplings (NACs) from the molecular orbital overlap files obtained in [step2](../../7_step2_cp2k/1_DFT). The NACs will be computed in Kohn-Sham states and also in both single-partcile (SP) and many-body (MB) excited state bases. 

## Table of contents
<a name="toc"></a>
1. [Importing needed libraries](#import)
2. [Overview of required files](#required_files)
3. [Computing NACs](#comp_nacs)     \
   3.1. [Kohn-Sham basis](#KS)\
   3.2. [Excited state basis](#excited_states)
   

### A. Learning objectives

* To be able to compute NACs in Kohn-Sham and excited state bases

### B. Use cases

* [Computing NACs](#comp_nacs)


### C. Functions

- `libra_py`
  - `workflows`
    - `nbra`
      - [`step3`](#comp_nacs)
        - [`run_step3_ks_nacs_libint`](#KS)
        - [`run_step3_sd_nacs_libint`](#excited_states)


## 1. Importing needed libraries <a name="import"></a>
[Back to TOC](#toc)

Import `step3` modules which is used to compute the NACs in Kohn-Sham basis using `run_step3_ks_nacs_libint` and in excited states basis using `run_step3_sd_nacs_libint`. 

In [1]:
import os
from libra_py.workflows.nbra import step3

/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: R

## 2. Overview of required files <a name="required_files"></a>
[Back to TOC](#toc)

* `../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res`

The MO overlap files are needed and stored in this folder.

* `../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles`

All of the logfiles obtained from the electronic structure calculations of CP2K. These files will be needed to find the Kohn-Sham HOMO index and also extract the the TD-DFT data for each step.

## 3. Computing NACs <a name="comp_nacs"></a>
[Back to TOC](#toc)

### 3.1. Kohn-Sham basis <a name="KS"></a>

The `libra_py.workflow.nbra.step3.run_step3_ks_nacs_libint(params)` computes the NACs between pairs of Kohn-Sham states using the molecular orbitals time-overlaps. 

Libra stores the overlap data in 2-spinor format as follows:

<div>
<img src="./active_space_0.png" width="500"/>
</div>

With no spin-orbit couplings, two blocks of the matrix is zero. Since most of the elements of the overlap matrices are zero, we use the `scipy.sparse` library for storing and loading them. 

The paramters for this function are as follows:

`params['lowest_orbital']`: The lowest orbital considered in the computation of the MO overlaps. This value is exactly the same
as in the `run_template.py` file in step2.

`params['highest_orbital']`: The highest orbital considered in the computation of the MO overlaps. This value is exactly the same
as in the `run_template.py` file in step2.

`params['num_occ_states']`: The number of occupied orbitals to be considered from HOMO to lower occupied states. This value is defined by user.

`params['num_unocc_states']`: The number of unoccupied orbitals to be considered from LUMO to higher unoccupied states. This value is defined by user.

The two values above are used to create an active space which then will be used to select the elements from the MO overlap and 
energy matrices. 

`params['use_multiprocessing']`: A boolean flag to use the multiprocessing library of Python or not.

`params['nprocs']`: The number of processors to be used for the calculations. Libra will use this only if the `params['use_multiprocessing']` 
is set to `True`.

`params['time_step']`: The time-step used in the calculations in `fs`.

`params['es_software']`: The name of the software package used to compute the electronic structure calculations. This will be used to generate the HOMO 
index of that system so it can build the active space.

`params['path_to_npz_files']`: The full path to the MO overlap files.

`params['logfile_directory']`: The full path to the folder where all the log files are stored.

`params['path_to_save_ks_Hvibs']`: The full path to the folder in which the NACs between the Konh-Sham states are stored.

`params['start_time']`: The start time-step.

`params['finish_time']`: The finish time-step.


After setting all the above paramters, the calculations are run using `step3.run_step3_ks_nacs_libint(params)`.


In [2]:
params_ks = {
              'lowest_orbital': 24-10, 'highest_orbital': 24+11, 'num_occ_states': 10, 'num_unocc_states': 10,
              'use_multiprocessing': True, 'nprocs': 12, 'time_step': 1.0, 'es_software': 'cp2k',
              'path_to_npz_files': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res',
              'logfile_directory': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles',
              'path_to_save_ks_Hvibs': os.getcwd()+'/res-ks-DFT',
              'start_time': 1200, 'finish_time': 1401,
            }

#### For KS states - Applying correction to KS overlaps and computing the NACs in KS space
step3.run_step3_ks_nacs_libint(params_ks)

Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step Computing orthonormalization of St matrices in step Computing orthonormalization of St matrices in step Computing orthonormalization of St matrices in step Computing orthonormalization of St matrices in step   1210   12201215 12001205 1230122512501235

1245
1240

1255






Done with stepDone with step  Done with step12151225  1245 . Elapsed time:  . Elapsed time:. Elapsed time:2.3932480812072754  2.3383429050445557Done with step
 
2.4245140552520751255Computing orthonormalization of St matrices in step
 Done with step Computing orthonormalization of St matrices in stepDone with stepC

Computing orthonormalization of St matrices in step 1301Computing orthonormalization of St matrices in step
 1291
Done with step 1295 . Elapsed time: 2.560281991958618
Computing orthonormalization of St matrices in step 1296
Done with step 1285 . Elapsed time: 2.874279260635376
Computing orthonormalization of St matrices in step 1286
Done with step 1310 . Elapsed time: 2.432307004928589
Computing orthonormalization of St matrices in step 1311
Done with step 1315 . Elapsed time: 2.752101421356201
Computing orthonormalization of St matrices in step 1316
Done with step 1271 Done with step. Elapsed time: Done with step1266 . Elapsed time: 2.3595409393310547 2.2620930671691895

 Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step Done with step 1281 . Elapsed time: 2.4521656036376953
12671276Computing orthonormalization of St matrices in step
   1272. Elapsed time:1282 
2.4219675064086914
Computing orthonormalization of St matrices in step 

Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step  13521357

Done with step 1366 . Elapsed time: 2.486802816390991
Computing orthonormalization of St matrices in step 1367
Done with step 1371 . Elapsed time: 2.9334447383880615
Computing orthonormalization of St matrices in step 1372
Done with step 1376 . Elapsed time: 2.988920211791992
Done with step 1327 . Elapsed time: Computing orthonormalization of St matrices in step 13772.514091968536377

Computing orthonormalization of St matrices in step 1328
Done with stepDone with step 1322 . Elapsed time:Done with step 1332 1347 . Elapsed time:  . Elapsed time:  2.6234319210052492.0775067806243896
Computing orthonormalization of St matrices in step2.4530792236328125

Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step  1333 1323

1348
Done with step 1337 . Elapsed time: 2.156080484390259
Computing orthonormalization of St matrices in step 

### 3.2. Excited state basis <a name="excited_states"></a>
[Back to TOC](#toc)

Below, we will be using `step3.run_step3_sd_nacs_libint` function to compute the time-overlaps and nonadiabatic couplings between excited states basis. This can be done either in the many-body or single-particle basis. A schematic of the workflow for selecting new active space is shown below:


Some parameters are common with the ones used to run `step3.run_step3_ks_nacs_libint(params)` above.

Other parameters needed to run the `step3.run_step3_sd_nacs_libint(params)` function are as follows:

`params['isUKS']`: A boolean flag for unrestricted spin calculations.

`params['is_many_body']`: If set to `True`, the NACs will be computed between pairs of many-body (TD-DFT) states. Also, the NACs between single-particle 
SDs obtained from the TD-DFT results will be computed as well. Otherwise, only single-particle NACs will be computed only for the SDs obtained from
`num_occ_states` and `num_unocc_states`. This will be used for xTB calculations in which no TD-DFT was performed.

`params['number_of_states']`: The number of TD-DFT states to consider. This value should not exceed the number of requested TD-DFT states in the CP2K
calculations.

`params['tolerance']`: A lower bound for selection of the excitation with configuration interaction coefficients higher than this value.

`params['verbosity']`: An integer value showing the printing level. The default is set to 0. Higher values will print more data on the terminal.

`params['sorting_type']`: After defining the SDs, Libra will sort them either based on `'energy'` or `'identity'`.


The NACs can also be computed between excited states. These include the single-particle and many-body bases which the latter is obtained from the
TD-DFT calculations. First, we need to compute the overlap between excited state Slater-determinants (SDs) then they will be used to compute the NACs
between them. For many-body states, the configuration interaction coefficietns will be used. We will consider both single-particle 
and many-body for DFT calculations but only single-particle for xTB.

There are different ways of defining the excited states SDs (the single-particle excited state basis). The first is through 
defining the `num_occ_states` and `num_unocc_states` in which Libra
will start making the SDs from all of the occupied states (starting from `HOMO-num_occ_states+1`) to all of the unoccupied states (ends
to `LUMO+num_unocc_states-1`). Also, if the unrestricted spin calculation flag is set to `True`, the SDs will be made for both alpha and beta spin channels. 

For example, if you want to build the electron-only excitation basis, you need to set `params['num_occ_states'] = 1` and set `params['num_unocc_states']`
to a value less than the number of unoccupied orbitals that was considered in the computation of overlaps. This will generate all the electron-only
excitation from HOMO to unoccupied states.

If the TD-DFT calculations has been done, then Libra will go over all log files and 
generate all the SDs used for all the steps and therefore the definition of these SDs is automatic and Libra will replace the `num_occ_states` and
`num_unocc_states` itself based on the SDs that were generated from the TD-DFT log files. 



In [3]:
params_mb_sd = {
          'lowest_orbital': 24-10, 'highest_orbital': 24+11, 'num_occ_states': 10, 'num_unocc_states': 10,
          'isUKS': 0, 'number_of_states': 10, 'tolerance': 0.01, 'verbosity': 0, 'use_multiprocessing': True, 'nprocs': 12,
          'is_many_body': True, 'time_step': 1.0, 'es_software': 'cp2k',
          'path_to_npz_files': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res',
          'logfile_directory': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles',
          'path_to_save_sd_Hvibs': os.getcwd()+'/res-mb-sd-DFT',
          'outdir': os.getcwd()+'/res-mb-sd-DFT', 'start_time': 1200, 'finish_time': 1401, 'sorting_type': 'identity',
         }

step3.run_step3_sd_nacs_libint(params_mb_sd)

sd_unique_basis is: [[[24, 25], 'alp'], [[24, 26], 'alp'], [[23, 25], 'alp'], [[22, 25], 'alp'], [[21, 25], 'alp'], [[23, 26], 'alp'], [[22, 26], 'alp'], [[24, 27], 'alp'], [[24, 29], 'alp'], [[21, 26], 'alp'], [[20, 25], 'alp'], [[24, 28], 'alp'], [[20, 26], 'alp'], [[21, 30], 'alp'], [[19, 26], 'alp'], [[19, 25], 'alp'], [[20, 27], 'alp'], [[23, 27], 'alp'], [[23, 28], 'alp'], [[24, 30], 'alp']]
sd_states_reindexed is: [[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 7, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 8, -6], [1, -1, 2, -2, 3, -3, 4, -4, 7, -5, 6, -6], [1, -1, 2, -2, 3, -3, 7, -4, 5, -5, 6, -6], [1, -1, 2, -2, 7, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 8, -5, 6, -6], [1, -1, 2, -2, 3, -3, 8, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 9, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 11, -6], [1, -1, 2, -2, 8, -3, 4, -4, 5, -5, 6, -6], [1, -1, 7, -2, 3, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 10, -6], 

Done with computing the SD overlap of step 9. Elapsed time 0.02121877670288086Done with computing the SD overlap of step 39. Elapsed time 0.0210647583007812565
Done with computing the SD overlap of step 34. Elapsed time 0.021899938583374023Done with computing the SD overlap of step 19. Elapsed time 0.021157503128051758

Done with computing the SD overlap of step 59. Elapsed time 0.021277904510498047
Computing the SD overlaps for step
Computing the SD overlaps for step Computing the SD overlaps for step
 80 75Computing the SD overlaps for step85 
Computing the SD overlaps for step90 
Computing the SD overlaps for step100

Done with computing the SD overlap of step 54. Elapsed time 0.02545022964477539 
95

Computing the SD overlaps for step 105
Done with computing the SD overlap of step 49. Elapsed time 0.021118879318237305
Computing the SD overlaps for step Done with computing the SD overlap of step 24. Elapsed time 0.021130084991455078
Computing the SD overlaps for step 115
110
Done wi

Done with computing the SD overlap of step 120. Elapsed time 0.02088761329650879
Computing the SD overlaps for step 121
Done with computing the SD overlap of step 125. Elapsed time 0.021355867385864258Done with computing the SD overlap of step 135. Elapsed time 0.021173477172851562Done with computing the SD overlap of step 130. Elapsed time 0.021245718002319336
Computing the SD overlaps for stepDone with computing the SD overlap of step 165. Elapsed time 0.022055625915527344 
131Done with computing the SD overlap of step 155. Elapsed time 0.02131342887878418
Computing the SD overlaps for step
Done with computing the SD overlap of step 145. Elapsed time 0.021111249923706055Computing the SD overlaps for step 
166Done with computing the SD overlap of step 170. Elapsed time 0.0211179256439209

Computing the SD overlaps for step
Done with computing the SD overlap of step 150. Elapsed time 0.02253890037536621Computing the SD overlaps for stepDone with computing the SD overlap of step 175. El

Computing the SD overlaps for stepDone with computing the SD overlap of step 183. Elapsed time 0.018587589263916016 
188Computing the SD overlaps for step
 184
Done with computing the SD overlap of step 192. Elapsed time 0.01934337615966797
Computing the SD overlaps for stepDone with computing the SD overlap of step 197. Elapsed time 0.0196225643157959 
193
Computing the SD overlaps for step 198
Done with computing the SD overlap of step 188. Elapsed time 0.017266273498535156
Computing the SD overlaps for step 189Done with computing the SD overlap of step 184. Elapsed time 0.01874995231628418

Done with computing the SD overlap of step 193. Elapsed time 0.01950526237487793
Computing the SD overlaps for stepDone with computing the SD overlap of step 198. Elapsed time 0.019690990447998047 
194
Computing the SD overlaps for step 199
Done with computing the SD overlap of step 189. Elapsed time 0.01788616180419922
Done with computing the SD overlap of step 194. Elapsed time 0.01951742172241